# **Interactive Visualization in Python**

## **General Structure for Interactive Visualizations**

Regardless of the library, an interactive plot follows the same workflow:

### **A. Define the data and the model**

1. Construct the domain (e.g., `x = np.linspace(...)`).
2. Define a function that transforms parameters into data (e.g., `y = f(x; parameters)`).

**Pseudocode:**

```python
# Step 1: data/model
x = ...
def compute_y(params):
    return f(x, params)
```

### **B. Create an initial figure**

1. Choose the plotting library.
2. Draw the initial dataset (usually with default parameter values).
3. Store a handle to the elements that will be updated (line object, data source, or trace).

**Pseudocode:**

```python
figure = make_figure()
plot_handle = figure.add_line(x, compute_y(default_params))
```

### **C. Connect UI controls to an update function**

Every interactive framework has a way to trigger updates:

* **Matplotlib** uses Python callbacks (`interact`, `observe`, etc.).
* **Bokeh** uses JS callbacks or Python server-side callbacks.
* **Plotly** uses Python callbacks with `FigureWidget` or built-in widget syncing.

The logic is always:

1. Users change a control (slider, dropdown, etc.).
2. A callback receives new parameter values.
3. The callback recomputes `y = f(x; parameters)`.
4. The visual element is updated in place.

**Pseudocode:**

```python
def update(params):
    new_y = compute_y(params)
    update_plot(plot_handle, new_y)

ui_control.on_change(update)
display(ui_control, figure)
```

### **D. Library-to-Library Mapping**

| Step              | Matplotlib (ipywidgets) | Bokeh (JS/Python callbacks)        | Plotly (FigureWidget)  |
| ----------------- | ----------------------- | ---------------------------------- | ---------------------- |
| Data definition   | Numpy arrays            | Numpy arrays → JS arrays           | Numpy arrays           |
| Figure creation   | `plt.plot(...)`         | `figure()`, `ColumnDataSource`     | `go.FigureWidget`      |
| Updatable element | `Line2D` object         | `ColumnDataSource`                 | `Scatter` trace        |
| Callback type     | Python function         | JavaScript (JS) or Python (server) | Python method callback |
| Update method     | mutate line data        | reassign `source.data`             | mutate trace `.y`      |

The conceptual pipeline is identical even though the APIs differ.

## **0. Common Setup Cell**

In [1]:
import numpy as np

## **1. Matplotlib + ipywidgets**

In [5]:
# Cell 1: interactive Matplotlib plot with ipywidgets
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

# x grid shared by all calls
x = np.linspace(0, 2 * np.pi, 400)

def plot_sine_matplotlib(freq=1.0, amplitude=1.0):
    """Draw amplitude * sin(freq * x)."""
    y = amplitude * np.sin(freq * x)
    plt.figure(figsize=(6, 4))
    plt.plot(x, y)
    plt.ylim(-2.1, 2.1)
    plt.xlabel("x")
    plt.ylabel("y")
    plt.title(f"{amplitude:.2f} × sin({freq:.2f}·x)")
    plt.grid(True)
    plt.show()

interact(
    plot_sine_matplotlib,
    freq=FloatSlider(min=0.5, max=2.0, step=0.5, value=1.0, description="freq"),
    amplitude=FloatSlider(min=0.5, max=2.0, step=0.5, value=1.0, description="amp"),
);

interactive(children=(FloatSlider(value=1.0, description='freq', max=2.0, min=0.5, step=0.5), FloatSlider(valu…

## **2. Bokeh (Pure-JS Slider Callback)**

In [9]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Slider, CustomJS
from bokeh.layouts import column
import numpy as np

output_notebook()

# Data source
x = np.linspace(0, 2 * np.pi, 400)
y = np.sin(x)
source = ColumnDataSource(data=dict(x=x, y=y))

# Figure
p = figure(width=600, height=300, title="Bokeh: sin(f·x)")
p.line("x", "y", source=source, line_width=2)

# Slider
slider = Slider(start=0.5, end=5.0, step=0.5, value=1.0, title="frequency")

# JS callback: build a *new* y array and reassign source.data
callback = CustomJS(
    args=dict(source=source),
    code="""
        const f = cb_obj.value;                // slider value
        const x = source.data.x;               // original x
        const y = new Array(x.length);         // new y array

        for (let i = 0; i < x.length; i++) {
            y[i] = Math.sin(f * x[i]);
        }

        // Reassign data so Bokeh sees the change and redraws
        source.data = { x: x, y: y };
    """
)

slider.js_on_change("value", callback)

layout = column(p, slider)
show(layout)

Loading BokehJS ...

## **3. Plotly (FigureWidget + ipywidgets)**

In [2]:
# Cell 3: interactive Plotly FigureWidget with ipywidgets
import plotly.graph_objects as go
import ipywidgets as widgets

x = np.linspace(0, 2 * np.pi, 400)

# Create a FigureWidget
fig = go.FigureWidget(
    data=[go.Scatter(x=x, y=np.sin(x), mode="lines")],
    layout=go.Layout(
        title="Plotly: sin(f·x)",
        xaxis=dict(title="x"),
        yaxis=dict(title="y", range=[-2.1, 2.1]),
        height=350,
        width=700,
    ),
)
curve = fig.data[0]

# Slider
freq_slider = widgets.FloatSlider(
    min=0.5,
    max=5.0,
    step=0.5,
    value=1.0,
    description="freq",
    continuous_update=True,
)

def update_plot(change):
    f = freq_slider.value
    with fig.batch_update():
        curve.y = np.sin(f * x)

freq_slider.observe(update_plot, names="value")

widgets.VBox([freq_slider, fig])

    'dat…